## Linear Optimization

We want to *maximize* `3x + y` subject to the following constraints

- 0 <= x <= 1
- 0 <= y <= 2
- x + y <= 2

References 
- https://developers.google.com/optimization/introduction/python

In [1]:
from ortools.init import pywrapinit
from ortools.linear_solver import pywraplp

In [2]:
# Create the linear solver with the GLOP backend.
solver = pywraplp.Solver.CreateSolver("GLOP")

In [3]:
# Create the variables
x = solver.NumVar(0, 1, "x")
y = solver.NumVar(0, 2, "y")

print("Number of variables =", solver.NumVariables())

Number of variables = 2


In [4]:
# Define the constraints.
# Create a linear constraint, 0 <= x + y <= 2.
ct = solver.Constraint(0, 2, "ct")
ct.SetCoefficient(x, 1)
ct.SetCoefficient(y, 1)

print("Number of constraints =", solver.NumConstraints())

Number of constraints = 1


In [5]:
# Define the objective function.
objective = solver.Objective()
objective.SetCoefficient(x, 3)
objective.SetCoefficient(y, 1)
objective.SetMaximization()

In [6]:
solver.Solve()
print("Solution:")
print("Objective value =", objective.Value())
print("x =", x.solution_value())
print("y =", y.solution_value())

Solution:
Objective value = 4.0
x = 1.0
y = 1.0


# Using cp_model

In [7]:
from ortools.sat.python import cp_model

In [8]:
class MaximizeSolutionPrinter(cp_model.CpSolverSolutionCallback):
    def __init__(self, x, y):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self._solution_count = 0
        self._x = x
        self._y = y

    def on_solution_callback(self):
        self._solution_count += 1
        print("Solution %i" % self._solution_count)
        print("x = {}, y = {}".format(self.Value(self._x), self.Value(self._y)))

In [9]:
model = cp_model.CpModel()
x = model.NewIntVar(0, 1, "x")
y = model.NewIntVar(0, 2, "y")

model.Add(0 <= x + y)
model.Add(x + y <= 2)
model.Maximize(3 * x + y)
solution_printer = MaximizeSolutionPrinter(x, y)

solver = cp_model.CpSolver()
solver.Solve(model, solution_printer) == cp_model.OPTIMAL

Solution 1
x = 1, y = 1


True

In [14]:
solver.ObjectiveValue(), solver.Value(x), solver.Value(y)

(4.0, 1, 1)

In [15]:
print(solver.ResponseStats())

CpSolverResponse summary:
status: OPTIMAL
objective: 4
best_bound: 4
booleans: 0
conflicts: 0
branches: 0
propagations: 0
integer_propagations: 0
restarts: 0
lp_iterations: 0
walltime: 0.004344
usertime: 0.004344
deterministic_time: 6.4e-07
gap_integral: 0



In [12]:
help(solver)

Help on CpSolver in module ortools.sat.python.cp_model object:

class CpSolver(builtins.object)
 |  Main solver class.
 |  
 |  The purpose of this class is to search for a solution to the model provided
 |  to the Solve() method.
 |  
 |  Once Solve() is called, this class allows inspecting the solution found
 |  with the Value() and BooleanValue() methods, as well as general statistics
 |  about the solve procedure.
 |  
 |  Methods defined here:
 |  
 |  BestObjectiveBound(self)
 |      Returns the best lower (upper) bound found when min(max)imizing.
 |  
 |  BooleanValue(self, literal)
 |      Returns the boolean value of a literal after solve.
 |  
 |  NumBooleans(self)
 |      Returns the number of boolean variables managed by the SAT solver.
 |  
 |  NumBranches(self)
 |      Returns the number of search branches explored by the solver.
 |  
 |  NumConflicts(self)
 |      Returns the number of conflicts since the creation of the solver.
 |  
 |  ObjectiveValue(self)
 |      Retu